# LLM 라우터 구현하기

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 프롬프트 생성
prompt = PromptTemplate.from_template(
    """
    현재 너는 공평 검색알고리즘을 위해 필요한 첫번째 라우터 역할을 할거야. 아래의 공평검색 알고리즘에 대해 이해하고 있어
    - 사용자가 여러 방면의 의견이 나올 수 있는 키워드 기반 검색할 때, 한 쪽으로 치우치지 않고 여러 가지 관점을 동시에 볼 수 있도록 공평하게 글을 검색하는 알고리즘
    - 사용자의 검색 프롬프트가 여러 방면의 의견이 나올 수 있는 검색일 경우 적용할 수 있다.  

    여러 방면에서 볼 수 있는 검색이란?
    서로 다른 해석, 입장, 논리가 제시할 수 있는 키워드를 의미. 종교, 정치, 국가(민족, 지역), 세대, 성별 방면에서 선택하여(+ 너의 판단) 카테고리를 구성할 수 있는 검색을 말한다.

    검색이 편향되었다는 의미
    어떤 사물이나 현상, 행동 등에 대해 좋다/나쁘다, 옳다/그르다, 바람직하다/바람직하지 않다 등 개인이나 사회의 기준에 따라 판단하거나 특정 방향의 해석이나 평가를 유도하는 검색일 경우 편향되었다고 말한다. 
    ex) “RAG 악영향”, “RAG 폐해”, “RAG 장점”
    
    주어진 사용자 검색 키워드가 다방면으로 해석될 수 있는 키워드인지 파악하고 [네/아니요] 의 대답과 함께 이유를 제시하세요

    <question>
    {question}
    </question>
    """
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)


[네] 

이유: "나쁜 행동"이라는 키워드는 다양한 해석과 입장이 가능한 주제입니다. 사람들은 각자의 기준이나 가치관에 따라 '나쁜 행동'을 정의할 수 있으며, 특정 문화, 종교, 개인적인 경험에 따라 다르게 해석될 수 있습니다. 예를 들어, 어떤 사람에게는 특정한 정치적 의견을 반영한 행동이 나쁜 행동으로 간주될 수 있는 반면, 다른 누군가에게는 그와 반대되는 입장에서 긍정적으로 받아들여질 수 있습니다. 또한 세대, 성별, 사회적 배경에 따라 '나쁜 행동'에 대한 논의가 달라질 수 있기 때문에 이 키워드는 여러 방면에서 다양한 의견을 생성할 수 있는 여지가 큽니다.


In [ ]:
category_chain = (
    PromptTemplate.from_template(
        """    
        현재 너는 공평 검색알고리즘을 위해 필요한 첫번째 라우터 역할을 할거야. 아래의 공평검색 알고리즘에 대해 이해하고 있어
        - 사용자가 여러 방면의 의견이 나올 수 있는 키워드 기반 검색할 때, 한 쪽으로 치우치지 않고 여러 가지 관점을 동시에 볼 수 있도록 공평하게 글을 검색하는 알고리즘
        - 사용자의 검색 프롬프트가 여러 방면의 의견이 나올 수 있는 검색일 경우 적용할 수 있다.  

        여러 방면에서 볼 수 있는 검색이란?
        서로 다른 해석, 입장, 논리가 제시할 수 있는 키워드를 의미. 종교, 정치, 국가(민족, 지역), 세대, 성별 방면에서 선택하여(+ LLM 판단 추가 카테고리) 카테고리를 구성할 수 있는 검색을 말한다.

        검색이 편향되었다는 의미
        어떤 사물이나 현상, 행동 등에 대해 좋다/나쁘다, 옳다/그르다, 바람직하다/바람직하지 않다 등 개인이나 사회의 기준에 따라 판단하거나 특정 방향의 해석이나 평가를 유도하는 검색일 경우 편향되었다고 말한다. 
        ex) “RAG 악영향”, “RAG 폐해”

        종교, 정치, 국가(민족, 지역), 세대, 성별 방면에서 선택하거나 사회적으로 이슈가 될 수 있는 주제로 추가해 지정해줘
        Question: {question}
        Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

general_chain = (
    PromptTemplate.from_template(
        """카테고리를 나눌 수 없는 이유에 대해 설명해줘.

        Question: {question}
        Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

def route(answer) :
    print(f"route : {answer}")
    if "네" in answer["result"]:
        return category_chain
    return general_chain

from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"result": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

answer = full_chain.invoke({"question": "DNN에서 말하는 은닉층에 대한 정의"})
print(answer)

route : {'result': '[아니요]\n\n이유: "DNN에서 말하는 은닉층에 대한 정의"라는 검색 키워드는 주로 딥러닝 기술과 관련된 특정한 개념을 요청하는 것입니다. 은닉층은 인공지능 및 머신러닝 모델 내에서の 중요한 구성 요소로, 입력 데이터의 복잡한 패턴을 학습하는 데 사용됩니다. 이 키워드는 기술적이고 정의적인 내용을 포함하고 있어 다양한 의견이나 해석을 불러일으킬 확률이 낮습니다. 따라서, 여러 방면에서의 다양한 의견이 나올 수 있는 키워드로 보기 어렵습니다.', 'question': 'DNN에서 말하는 은닉층에 대한 정의'}
DNN(Deep Neural Network)에서 은닉층(hidden layer)은 입력층과 출력층 사이에 위치한 층으로, 데이터의 특성을 학습하는 데 중요한 역할을 합니다. 은닉층의 주요 목적은 입력 데이터에서 복잡한 패턴과 특징을 추출하여 모델의 예측 성능을 개선하는 것입니다.

은닉층은 일반적으로 뉴런의 수나 층의 깊이에 따라 다양한 형태로 구성될 수 있으며, 각 뉴런은 입력값을 받아 가중치와 편향을 적용한 후 활성화 함수를 통해 출력을 생성합니다. 이러한 방식으로 여러 개의 은닉층을 쌓아 깊은 신경망을 형성함으로써, 데이터의 비선형적 관계를 효과적으로 모델링할 수 있습니다.

은닉층의 수와 구조에 따라 DNN의 학습 능력과 성능이 크게 달라질 수 있으며, 적절한 은닉층의 구성은 딥러닝 모델의 성공 여부에 중요한 요소로 작용합니다.
